In [1]:
import pathlib
import pandas as pd
import seedir as sd

### Parse Info
This function parses sniffles column 8 (INFO) to extract the Type, End, Lenght, and Chr2 of the Strcutural variant. 

Sample data at index 7:
```
PRECISE;SVMETHOD=Snifflesv1.0.12;CHR2=1;END=3097326;ZMW=17;STD_quant_start=0.000000;STD_quant_stop=1.897367;Kurtosis_quant_start=2.000000;Kurtosis_quant_stop=1.946987;SVTYPE=INS;SUPTYPE=AL;SVLEN=3644;STRANDS=+-;STRANDS2=9,8,9,8;RE=17;REF_strand=14,14;Strandbias_pval=1;AF=0.377778
```

In [2]:
def parse_info(line_array, infoIndex):
    
    dict_values={'SVTYPE':'NN', 'END':-1, 'CHR2':'', 'SVLEN':0}
    
    if(len(line_array)<=infoIndex):
        return dict_values
    
    infoString=line_array[infoIndex]
    info_array = infoString.split(";");
    
    if(len(info_array)<=2):
        return dict_values
    
    for k, v in dict_values.items():
        for item in info_array:
            if(k+"=" in item):
                dict_values[k]=item.replace(k+"=","")
                break
    
    return dict_values

### Generate BED File

The output file has the following format:

Column | Data         | Comment                          
:---   | ---          | ---                              
Col1   | chr num      | 'chr' prefixed                   
Col2   | start        | from sniffles output column (2)*  
Col3   | end          | fom info data *                    
Col4   | length       | fom info data              

In case both start and end are the same (usually insertions), the start-end range will be modified by 20 bp at the start and end.

In [8]:
def bed_from_vcf(strain_name, input_file, final_dir, custom_mappings={}, min_len=-1, unified_file=False,
                 split_precise=False, fix_coordinates=True, exclude_bnd=True, exlude_noend=True):

    """
    bed_from_vcf(strain_name, input_file, final_dir, custom_mappings, unified_file, split_precise, 
                 fix_coordinates, exclude_bnd, exlude_noend)

    Generates BED Files from VCF files (Currently supports Sniffles and PBSV).
    
    Parameters
    ----------
    strain_name : str
        Name prefix for generated files
    input_file : str
        Input VCF
    final_dir : str
        Output Dir
    custom_mappings : dict
        Custom mapping for SV Type names
    min_len: int
        Minimum length for SVs (exclude SVs with length lower than). Set -1 to report all lengths.
    unified_file: bool
        A file is generated per SV Type unless this parameter is true.
    split_precise, fix_coordinates, exclude_bnd, exlude_noend : bool
        Whether or not the output will differentiate precise from non-precise SVs,
        fix coordinates so start is always lower than end, and exclude entries 
        with SVTYPE=BND and no end position
    
        
    Returns
    -------
    list
        Merged list of SV Types found
    """
    
    #chr_names=["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chrX","chrY"]
    chr_names=["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","X"]
    nucleotides = ["A", "T", "G", "C"]

    type_dicts = {}
    all_types = []
    
    with open(input_file, "r") as vcf_file:
        for line in vcf_file:
            line_array=line.strip().split("\t")

            if(len(line_array) <8 or line.startswith("#")):
                continue

            chr1=line_array[0]
            start=int(line_array[1])-1
            info=parse_info(line_array, 7)

            end=int(info['END'])-1
            length=abs(int(info['SVLEN']))
            
            if min_len > 0 and length < min_len :
                continue

            sv_type = info['SVTYPE'].replace("/","").upper()
            sv_type = custom_mappings.get(sv_type, sv_type)
            
            if unified_file:
                sv_type =  "ALL"

            if((exclude_bnd and info['SVTYPE']=='BND') or (chr1 not in chr_names) or (exlude_noend and info['END']==-1)):
                continue

            if fix_coordinates and start > end:
                aux = start
                start = end
                end = aux

            if fix_coordinates and start == end:
                start -= 20
                end += 20

            if split_precise:
                if any(e in str(line_array[4]) for e in nucleotides):
                    sv_type += "_precise"
                else:
                    sv_type += "_other"


            type_entries = type_dicts.get(sv_type, [])
            type_entries.append({"chr": chr1, "start": start, "end": end, "len": str(length)}) 

            type_dicts[sv_type]=type_entries
        
        pathlib.Path(final_dir).mkdir(parents=True, exist_ok=True)
        
        for type_dict in type_dicts.keys():
                type_df = pd.DataFrame(type_dicts[type_dict])
                type_df.to_csv(final_dir + type_dict + "_" + strain_name + ".bed", sep='\t', index=False, header=False)

        all_types = list(set(all_types) | set([*type_dicts]))
        
    return all_types

In [14]:
# sv_file_path = '../../data/DBA2J/calls/r64089e.minimap2_sniffles.vcf'
# final_dir = '../../data/DBA2J/bed/50/minimap2-sniffles/'

# sv_file_path = '../../data/DBA2J/calls/r64089e-ngmlr_sniffles.vcf'
# final_dir = '../../data/DBA2J/bed/50/ngmlr-sniffles/'
 
sv_file_path = '../../data/DBA2J/calls/r64089e-minimap2_pbsv.vcf'
final_dir = '../../data/DBA2J/bed/50/minimap2-pbsv/'

custom_mappings={
    "DEL": "DEL",
    "DELINS": "DEL",
    "DELLINKED": "DEL",
    "INS": "INS",
    "INSLINKED": "INS",
    "INSLINKEDINS": "INS",
    "INV": "INV",
    "INVDELINS": "INV",
    "INVINS": "INV",
    "INVDUP": "INV",
    "INVDEL": "INV",
    "DELINV": "INV",
    "DUP": "DUP",
    "GAIN": "DUP",
    "DUPINS": "DUP",
    "CNV": "DUP",
    "TANDEMDUP": "DUP",
    "TANDEMDUPINV": "DUP",
    "TANDEMLOWDUP": "DUP",
}

bed_from_vcf("DBA2J", sv_file_path, final_dir, custom_mappings, 50)

['INV', 'DEL', 'DUP', 'INS']

In [15]:
sd.seedir("../../data/DBA2J/bed/50/", style='emoji', itemlimit=10, depthlimit=3, include_files='.*\.bed$', regex=True, sort=True)

📁 50/
├─📁 intersects/
│ ├─📄 DEL_catalog_vs_H1.bed
│ ├─📄 DEL_catalog_vs_H2.bed
│ ├─📄 DEL_catalog_vs_validated.bed
│ ├─📄 DEL_pbsv_vs_H1.bed
│ ├─📄 DEL_pbsv_vs_H2.bed
│ ├─📄 DEL_pbsv_vs_catalog.bed
│ ├─📄 DEL_pbsv_vs_validated.bed
│ ├─📄 INS_catalog_vs_H6.bed
│ ├─📄 INS_catalog_vs_H7.bed
│ └─📄 INS_catalog_vs_validated.bed
├─📁 minimap2-pbsv/
│ ├─📄 DEL_DBA2J.bed
│ ├─📄 DUP_DBA2J.bed
│ ├─📄 INS_DBA2J.bed
│ └─📄 INV_DBA2J.bed
├─📁 minimap2-sniffles/
│ ├─📄 DEL_DBA2J.bed
│ ├─📄 DUP_DBA2J.bed
│ ├─📄 INS_DBA2J.bed
│ └─📄 INV_DBA2J.bed
└─📁 ngmlr-sniffles/
  ├─📄 DEL_DBA2J.bed
  ├─📄 DUP_DBA2J.bed
  ├─📄 INS_DBA2J.bed
  └─📄 INV_DBA2J.bed
